In [1]:
import sys
import warnings
warnings.filterwarnings('ignore')
import glob
!{sys.executable} -m pip install numpy pandas matplotlib scikit-learn pyarrow seaborn imblearn xgboost| grep -v 'already satisfied'

#

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

'grep' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
df = pd.read_csv("train.csv")

In [3]:
df.head()

,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status
0,0,999,D-penicillamine,21532,M,N,N,N,N,2.3,316.0,3.35,172.0,1601.0,179.80,63.0,394.0,9.7,3.0,D
1,1,2574,Placebo,19237,F,N,N,N,N,0.9,364.0,3.54,63.0,1440.0,134.85,88.0,361.0,11.0,3.0,C
2,2,3428,Placebo,13727,F,N,Y,Y,Y,3.3,299.0,3.55,131.0,1029.0,119.35,50.0,199.0,11.7,4.0,D
3,3,2576,Placebo,18460,F,N,N,N,N,0.6,256.0,3.50,58.0,1653.0,71.30,96.0,269.0,10.7,3.0,C
4,4,788,Placebo,16658,F,N,Y,N,N,1.1,346.0,3.65,63.0,1181.0,125.55,96.0,298.0,10.6,4.0,C


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7905 entries, 0 to 7904
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             7905 non-null   int64  
 1   N_Days         7905 non-null   int64  
 2   Drug           7905 non-null   object 
 3   Age            7905 non-null   int64  
 4   Sex            7905 non-null   object 
 5   Ascites        7905 non-null   object 
 6   Hepatomegaly   7905 non-null   object 
 7   Spiders        7905 non-null   object 
 8   Edema          7905 non-null   object 
 9   Bilirubin      7905 non-null   float64
 10  Cholesterol    7905 non-null   float64
 11  Albumin        7905 non-null   float64
 12  Copper         7905 non-null   float64
 13  Alk_Phos       7905 non-null   float64
 14  SGOT           7905 non-null   float64
 15  Tryglicerides  7905 non-null   float64
 16  Platelets      7905 non-null   float64
 17  Prothrombin    7905 non-null   float64
 18  Stage   

In [5]:
# create train and validation sets that are divided equally

In [6]:
from sklearn.model_selection import train_test_split

train, validation = train_test_split(df, stratify=df["Status"], test_size=0.2)

In [7]:
numeric_values = ["N_Days","Age","Bilirubin","Cholesterol","Albumin","Copper","Alk_Phos","SGOT","Tryglicerides","Platelets","Prothrombin","Stage"]

In [8]:
categorical_values = ["Drug","Sex","Ascites","Hepatomegaly","Spiders","Edema"]
categorical_bin_values = ["Drug","Sex","Ascites","Hepatomegaly","Spiders"]
categorical_oh_values = ["Edema"]

In [9]:
features_to_scale = ["N_Days","Age","Bilirubin","Cholesterol","Albumin","Copper","Alk_Phos","SGOT","Tryglicerides","Platelets","Prothrombin","Stage"]

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder

# numeric pipeline
num_pipeline = Pipeline([('func_transformer', FunctionTransformer(np.log1p)), 
                         ('minmax_scaler', MinMaxScaler()), ])




In [11]:
from sklearn.base import BaseEstimator, TransformerMixin 
class BinaryEncoder (BaseEstimator, TransformerMixin):
    def __init__ (self, features):
        self.features = features
    def fit(self, X, y=None): 
        return self # do nothing
    def transform(self, X):
        for c in self.features:
            values = X[c].unique()
            X[c].replace(values,(0,1), inplace=True)
        return X
        
        

In [12]:
# full pipeline (numeric & categorical)
full_pipeline = ColumnTransformer([ ("num", num_pipeline, numeric_values), 
                                   ("bin_cat", BinaryEncoder(categorical_bin_values), categorical_bin_values), 
                                   ("cat", OneHotEncoder(sparse_output=False), categorical_oh_values), ]) 
train_prepared = full_pipeline.fit_transform(train)

In [13]:
# transform with pipeline
train_prepared = full_pipeline.fit_transform(train)

In [14]:
# Label Encoding
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
labels_en = label_encoder.fit_transform(train["Status"])
# k-fold (e.g. 5) validation, with log loss, scores need to be un-negated back
# scores = cross_val_score(model, train_prepared, labels_en, scoring="neg_log_loss", cv=5)


In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE


In [24]:
##Credit to James!

# Initialize SMOTE
smote = SMOTE(random_state=42)

# Apply SMOTE to the training data
X_train_smote, y_train_smote = smote.fit_resample(train_prepared, train["Status"])

# Retrain the classifier using the SMOTE-enhanced data
rf_smote = RandomForestClassifier(random_state=42)
rf_smote.fit(X_train_smote, y_train_smote)

RandomForestClassifier(random_state=42)

In [38]:
#initialise variables for validation sets
X_val = full_pipeline.fit_transform(validation)
y_val = validation["Status"]

y_val_pred_rf_smote = rf_smote.predict(X_val)

accuracy_val_rf_smote = accuracy_score(y_val, y_val_pred_rf_smote)
classification_report_val_rf_smote = classification_report(y_val, y_val_pred_rf_smote)

print('accuracy_val_rf_smote:\n',accuracy_val_rf_smote,'\n\n','classification_report_val_rf_smote:\n', classification_report_val_rf_smote)

accuracy_val_rf_smote:
 0.7773561037318153 

 classification_report_val_rf_smote:
               precision    recall  f1-score   support

           C       0.88      0.80      0.84       993
          CL       0.24      0.22      0.23        55
           D       0.68      0.80      0.73       533

    accuracy                           0.78      1581
   macro avg       0.60      0.60      0.60      1581
weighted avg       0.79      0.78      0.78      1581



In [33]:
from sklearn.model_selection import cross_val_score

K = 10

randomforest_scores = cross_val_score(
    rf_smote, X_train_smote, y_train_smote,
    scoring="neg_log_loss", cv=K)

In [36]:
def display_scores(scores):
    print("Scores:", scores) 
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

In [37]:
display_scores(randomforest_scores)

Scores: [-0.40734589 -0.36126788 -0.35016542 -0.34049781 -0.36823386 -0.31446077
 -0.31733695 -0.34123616 -0.31726728 -0.32025159]
Mean: -0.34380636106572665
Standard deviation: 0.027960351996320357
